## Creating a data generator to build our insections

In [27]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import os
load_dotenv()

True

### Environment variables

In [28]:
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("API_BASE_URL")

## DeepSeek AI

In [29]:
class DeepseekAI():
    
    def __init__(self,system_prompt,user_prompt,API_KEY, BASE_URL ):
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        self.api_key= API_KEY
        self.base_url= BASE_URL

    def __str__(self):
        print(self.system_prompt)
    
    def run_AI(self):
        try:
            client = OpenAI(
                api_key = self.api_key,
                base_url = self.base_url
            )

            messages = [
                {"role": "system", "content" : self.system_prompt},
                {"role": "user", "content" : self.user_prompt}
            ]

            response = client.chat.completions.create(
                model="deepseek-chat",
                messages= messages, # type: ignore
                response_format= {
                    'type': 'json_object'
                },
                temperature = 0.1,
                extra_body={
                    "thinking": {
                        "type": "enabled"
                        }
                    }

            )

            return json.loads(response.choices[0].message.content) # type: ignore
        except Exception as error:
            print(f"Error Deepseek-AI: {error}")

### Creating Values from each schema

In [30]:
# Country: Choose a random country, in order to get names, phones, addresses more realistic, for you data generation.
country = "Venezuela"

1. Users

In [31]:
# number of users per role
admin = 2
technician = 14
operator = 6
# AI prompts
system_prompt = """
You are an excellent JSON data generation assistant, you are going to create a list of users.

The schema will be:
- Roles & Rates: **Admin** (10.00 and 25.00) | **Operator** (8.00 and 15.00) | **Technician** (15.00 and 40.00).
- Country: Most popular first names and last names for that country.
- Address: Real city-based addresses from that country (street names, states).
- email: Based on names (e.g., juanperez@example.com), possibly with a number if needed for uniqueness.
- id: Generate a UUID-like string, 'USER-xxxx-XX' (e.g., 'USER-1234-AB').
- phone: Generate a unique phones based on country (+country_code-4XX-XXX-XXXX, eg: +58-400-000-0000).
- gender: Assign randomly but reasonably based on name if known, else random.
- password: Use "hashed_password_123" for all..
- created_at and updated_at: Random dates within the last 3 years, with updated_at possibly equal to or later than created_at.

JSON OUTPUT: [ {
      "id": "string",
      "first_name": "string",
      "last_name": "string",
      "email": "string",
      "phone": "string",
      "gender": "string",
      "address": "string",
      "password": "string,
      "role": "string",
      "hourly_rate": "float",
      "is_active": "boolean",
      "created_at": "datetime", 
      "updated_at": "datetime"
    },... ]

Output ONLY the JSON array (must start with `[` and end with `]`). No markdown, no explanations, no comments.

"""
user_prompt = f"""
Hi there! Please help me generate a list of users for a database schema based on the following details: 
I need a total of {admin + technician + operator} users, distributed as follows:
- Admins: {admin},
- Technicians: {technician} Make balances but almost them must be men, Exactly 4 must have `is_active: false`,
- Operators: {operator} Gender balance (male/female) is acceptable. Exactly 2 must have `is_active: false`.

The users should be from {country}. Please ensure that the generated data adheres to the specified roles and inlcudes realistic names, email addresses, email domains randomly: @gmail.com, @hotmail.com, @outlook.com.

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""

users_list = []
def create_user_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating user: {error}")

In [32]:
users_list: dict = create_user_insertions() # type: ignore

2. Client

In [36]:
# Number of clients to generate by type:
residential = 30
commercial = 15
# Prompts
system_prompt = """
You are an excellent JSON data generation assistant, you are going to create a list of Clients:

Data:
- Client_type: Residential | Commercial
- Country: Most popular first names and last names for that country.
- Address: Real city-based addresses from that country (street names, states).
- email: Based on names (e.g., juanperez@example.com), possibly with a number if needed for uniqueness.
- id: Generate a UUID-like string 'CLIENT-xxxx-[A-Z]' (eg: CLIENT-1245-EF).
- phone: Generate a unique phones based on country (+country_code-4XX-XXX-XXXX, eg: +58-400-000-0000).
- gender: Assign randomly but reasonably based on name if known, else random.
- created_at and updated_at: Random dates within the last 3 years, with updated_at possibly equal to or later than created_at.

JSON OUTPUT: [ {
      "id": "string",
      "first_name": "string",
      "last_name": "string",
      "email": "string",
      "phone": "string",
      "gender": "string",
      "address": "string",
      "client_type": "string",
      "created_at": "datetime", 
      "updated_at": "datetime"
    },...]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = f"""
Hi there! Please help me generate a list of clients for a database schema based on the following details: 
Generate exactly {residential + commercial} clients:
- Residential: {residential}
- Commercial: {commercial}

The client should be from {country}. Please ensure that the generated data adheres to the specified roles and inlcudes realistic names, email addresses, email domains randomly: @gmail.com, @hotmail.com, @outlook.com.

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
clients_list = []
def create_client_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating clients: {error}")

In [37]:
clients_list: dict = create_client_insertions()# type: ignore

3. Maintenance Types

In [40]:
system_prompt = '''You are an excellent JSON data generation assistant, you are going to create a list of maintenance types:

Field Specifications:
1. Include 15 diverse maintenance types across all three codes
2. For PM: frequency_days should be reasonable (30, 90, 180, 365, etc.)
3. For PdM: frequency_days should be condition-based but provide typical intervals
4. For CM: frequency_days should be null (unscheduled/emergency)

Constraints:
- code: 'PM' = Preventive Maintenance (regular scheduled maintenance) | 'CM' = Corrective Maintenance (repairs after failure) | 'PdM' = Predictive Maintenance (condition-based maintenance)
- Generate realistic maintenance types for both Residential and Commercial HVAC/AC systems.

JSON OUTPUT: [ {
    "id": "string (format: MT-001, MT-002, etc.)",
    "code": "string (PM, CM, or PdM only)",
    "name": "string (Clear, industry-standard HVAC maintenance name.)",
    "frequency_days": "integer or null (null for CM, number for PM/PdM)",
    "description": "string (Specific and actionable details of the task)"
  },...]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.

'''
user_prompt = """Generate comprehensive maintenance types for AC/HVAC systems covering all maintenance categories.

Make sure:
- PM types have regular frequency intervals
- CM types have null frequency (unscheduled)
- PdM types have appropriate monitoring intervals
- All IDs follow MT-XXX format
- Names are clear and professional
- Descriptions explain what technicians actually do

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments."""

maintenance_types_list = []
def create_maintenance_types_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating maintenance types: {error}")

In [41]:
maintenance_types_list: dict = create_maintenance_types_insertions()# type: ignore

4. Failure Types

In [42]:
system_prompt = """You are an excellent JSON data generation assistant, you are going to create a list of failure types:

CATEGORY GUIDANCE:
- 'Electrical': Wiring, capacitors, motors, transformers, circuit boards
- 'Mechanical': Bearings, belts, pulleys, fans, compressors, physical damage
- 'Refrigeration': Refrigerant leaks, compressor issues, coil problems, charge issues
- 'Airflow': Dirty filters, blocked vents, duct issues, fan problems
- 'Control System': Thermostats, sensors, actuators, communication issues
- 'Drainage': Clogged drains, pump failures, overflow issues
- 'Noise/Vibration': Excessive noise, vibration, rattling sounds
- 'Other': Environmental, installation, seasonal, or miscellaneous issues

SEVERITY GUIDANCE:
- 'Low': Minor issues, no immediate threat to operation (1-2 repair hours)
- 'Medium': Affects performance/efficiency but system still runs (2-4 repair hours)
- 'High': Significant performance impact or safety concern (4-8 repair hours)
- 'Critical': Complete system failure or safety hazard (8+ repair hours)

Field Specifications:
1. Generate 18 diverse failure types covering all 8 categories
2. Include both Residential and Commercial AC/HVAC systems examples
3. Make names specific and actionable (e.g., "Capacitor Failure" not "Electrical Problem")
4. Ensure estimated_repair_hours are realistic for the severity
5. Common_causes should list specific reasons (not vague descriptions)
6. Created_at dates should be from 2022 all of them
7. All field names must match the table column names exactly

JSON OUTPUT: [ {
    "id": "string (format: FT-001, FT-002, etc.)",
    "category": "string (must be one of the 8 categories)",
    "name": "string (specific, descriptive failure name)",
    "severity": "string (Low, Medium, High, or Critical)",
    "estimated_repair_hours": integer (1-12, realistic for the issue),
    "common_causes": "string (list 2-3 common causes)",
    "is_active": boolean (true),
    "created_at": "datetime string (YYYY-MM-DD HH:MM:SS, from 2022)"
  },... ]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = """Generate comprehensive failure types for AC/HVAC systems covering all 8 categories. Include a good mix of common residential and commercial failures with realistic repair times and severity levels.

Make sure:
- Each category has at least 2-4 examples
- Severity levels are appropriately assigned
- Repair hours match industry standards (e.g., capacitor = 1-2 hrs, compressor = 6-8 hrs)
- Names are specific and diagnostic (technician would use this term)
- Common causes list actual technical reasons
- 20%, of records have is_active: false (discontinued or obsolete)

Focus on realistic, actionable failure types that technicians actually diagnose and repair.

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments."""

failure_types_list = []
def create_failure_types_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating failures types : {error}")

In [43]:
failure_types_list: dict = create_failure_types_insertions()#type: ignore

5. Clients Equipment

* Get Clients info

In [44]:
def get_clients_id(client):
    try:
        return {
            "id": client["id"],
            "type": client["client_type"]
        }
    except Exception as error:
        print(f"Error getting clients info: {error}")

client_info = [ get_clients_id(x) for x in clients_list] # type: ignore

In [46]:
system_prompt = """You are an excellent JSON data generation assistant, you are going to create a list of client equipment, Generate data based on provided client IDs and client types.

EQUIPMENT TYPE DETAILS:
- 'Split AC': Indoor + outdoor units, common for residential/rooms
- 'Window AC': Single unit, through-wall installation
- 'Central AC': Ducted system for whole building
- 'Furnace': Heating system (gas, electric, oil)
- 'Heat Pump': Heating + cooling system
- 'Ventilation': ERV, HRV, exhaust fans
- 'Chiller': Large commercial cooling
- 'Other': Dehumidifiers, air purifiers, etc.

BRAND EXAMPLES:
- Residential: Carrier, Trane, Lennox, Goodman, York, Rheem, Daikin, Mitsubishi, LG
- Commercial: Trane, Carrier, York, McQuay, Daikin, Mitsubishi Electric, LG

IMPORTANT TIME CONSTRAINTS:
1. All dates must start from 2022 to present
2. Warranty must be exactly 1 year after installation (365 days)
3. installation_date format: YYYY-MM-DD
4. warranty_expiration = installation_date + 365 days
5. created_at should be close to installation_date (±15 days)
6. updated_at should be = created_at

Flied Spesifications:
1. Assign equipment logically based on client_type (Residential/Commercial)
2. Use realistic capacity_btu ranges for each equipment_type
3. Mix statuses: 60% Operational, 35% Requires Repair, 5% other
4. Ensure serial numbers are unique when provided
5. Create diverse equipment across all allowed types
6. Match commercial brands with commercial equipment

JSON OUTPUT: [
  {
    "id": "string (format: EQ-001, EQ-002, etc.)",
    "client_id": "string (from provided client IDs)",
    "equipment_name": "string (descriptive name)",
    "brand": "string",
    "model": "string (realistic model numbers)",
    "serial_number": "string (optional, format: SN-XXXXX-YY)",
    "equipment_type": "string (must match allowed values)",
    "capacity_btu": integer (Between realistic ranges based on equipment_type),
    "location": "string (specific room/location)",
    "installation_date": "string (YYYY-MM-DD)",
    "warranty_expiration": "string (YYYY-MM-DD, exactly 1 year later)",
    "status": "string (Operational, Requires Repair, Out of Service, Under Maintenance)",
    "description": "string (additional details)",
    "created_at": "string (YYYY-MM-DD HH:MM:SS)",
    "updated_at": "string (YYYY-MM-DD HH:MM:SS)"
  },..
]
Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = f"""I need to generate equipment records for {len(client_info)} clients. Here are the client IDs and their types:

clients' Information:
clients: {client_info}

Generate equipment records with these guidelines:
1. Residential clients: 1-3 pieces of equipment each (mostly Split AC, Window AC, Furnace)
2. Commercial clients: 2-4 pieces of equipment each (Central AC, Chiller, Ventilation, Heat Pumps)
3. All dates must be within 2022 to present
4. Warranty exactly 1 year from installation
5. Include realistic models, capacities, and locations
6. Add descriptive details in description field
7. Some equipment should have Requires Repair status (age appropriately)

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments."""

client_equipment_list = []
def create_client_equipment_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating client_equipment: {error}")

In [47]:
client_equipment_list: dict = create_client_equipment_insertions() #type: ignore

6. Maintenance Orders

* Get Users ID

In [57]:
technician_id = [ x.get("id", "error: Not Found") for x in users_list if x.get("role", "").lower() == "technician" and x.get("is_active", False) is True] # type: ignore

* Maintenance Types ID

In [58]:
maintenance_types_id = [x.get("id", "error: Not Found") for x in maintenance_types_list]# type: ignore

* Get Equipment by clients

In [62]:
def equipment_by_clients(clients, equipments_id):
    equipment_by_client = {}
    client_types = {}  

    for client in clients:
        client_id = client.get("id")
        client_type = client.get("type")
        
        if client_id:
            equipment_by_client[client_id] = []
            client_types[client_id] = client_type

    for eq in equipments_id:
        client_id = eq.get("client_id")
        equipment_id = eq.get("id")

        if client_id and client_id in equipment_by_client:
            equipment_by_client[client_id].append(equipment_id)

    data = []
    for client_id, eq_list in equipment_by_client.items():
        client_type = client_types.get(client_id, "Unknown")
        data.append({
            "Client_id": client_id,
            "Equipment_ids": eq_list,
            "client_type": client_type 
        })
    
    return data

* Getting data for create maintenance order data

In [63]:
all_equipments_by_clients = equipment_by_clients(client_info, client_equipment_list) # type: ignore

In [66]:
system_prompt = """You are an excellent JSON data generation assistant, you are going to create a list of maintenance order data with the following constraints:

IMPORTANT RELATIONSHIPS:
- client_id → clients.id (Residential or Commercial clients)
- equipment_id → client_equipment.id (Equipment owned by client)
- type_id → maintenance_types.id (PM, CM, or PdM maintenance types)
- user_id → users.id (Technicians performing the work)

BUSINESS RULES:
1. STATUS LOGIC:
   - 'scheduled': Only scheduled_date set, no completion/ratings
   - 'in_progress': scheduled_date passed, not completed yet
   - 'completed': completed_date set, ratings/costs filled
   - 'cancelled': No completion, may have notes

2. DATE LOGIC:
   - scheduled_date: Random dates within last 2 years
   - completed_date: For 'completed' status only, after scheduled_date
   - next_maintenance_date: For PM types only, 30-365 days after completion, add posible maintenance to next year
   - created_at: Close to when order was created
   - updated_at: ≥ created_at, often after completion

3. COST LOGIC:
   - labor_hours: 1-8 hours (PM=1-3, CM=2-6, PdM=1-2)
   - labor_cost: labor_hours × hourly_rate (technician rate)
   - parts_cost: $0-$500 (PM=$0-50, CM=$50-500)
   - total_cost: labor_cost + parts_cost

4. RATING & FEEDBACK:
   - Only for 'completed' status
   - Ratings: 3-5 for good service, 1-2 for issues
   - Feedback: Realistic customer comments

5. NOTES & FEEDBACK:
   - technician_notes: Technical details, findings, recommendations
   - customer_feedback: Customer comments on service quality

Field Spesification:
1. Ensure referential integrity (IDs must exist in related tables)
2. Status progression makes sense (can't complete before scheduled)
3. Costs calculated logically based on maintenance type
4. Realistic dates (no future completed dates)
5. Mix of statuses: 40% completed, 30% scheduled, 20% in_progress, 10% cancelled
6. For 'completed', 80% should have ratings/feedback
7. For PM types, include next_maintenance_date

JSON OUTPUT: [ {
    "id": "MO-001",
    "client_id": "string",
    "user_id": "string",
    "equipment_id": "string",
    "type_id": "string",
    "status": "string (scheduled/in_progress/completed/cancelled)",
    "scheduled_date": "YYYY-MM-DD",
    "completed_date": "YYYY-MM-DD or null",
    "technician_notes": "string or null",
    "customer_feedback": "string or null",
    "customer_rating": "integer (1-5) or null",
    "labor_hours": "decimal",
    "labor_cost": "decimal",
    "parts_cost": "decimal",
    "total_cost": "decimal",
    "next_maintenance_date": "YYYY-MM-DD or null",
    "created_at": "YYYY-MM-DD HH:MM:SS",
    "updated_at": "YYYY-MM-DD HH:MM:SS"
  },..]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = f"""Generate 50 maintenance orders with realistic data.

Data you will need to create relationship.
- From this {all_equipments_by_clients} get Client_id, equipment_id adn client_type in order to be realistic creating table
- User IDs (Technicians): {technician_id}, they can have between 2-3 orders
- Maintenance Type IDs: get from this maintenance y its code in order you can be more realistic {maintenance_types_list} 

GUIDELINES:
1. Match equipment to correct client
2. Assign technicians (user_id) from the provided technician list
3. Match maintenance types appropriately:
   - PM types for scheduled/preventive maintenance
   - CM types for corrective/repair orders
   - PdM types for diagnostic/monitoring orders
4. Create realistic date sequences:
   - scheduled_date within last 2 years
   - completed_date 1-14 days after scheduled_date for 'completed'
   - created_at 1-5 days before scheduled_date
5. Calculate costs realistically:
   - Use technician hourly rates (approx $20-40/hr for residential, $30-60/hr for commercial)
   - Parts costs: $0-50 for PM, $50-500 for CM, $0-20 for PdM
6. Include detailed technician notes for completed orders
7. Add customer feedback and ratings (mostly positive 4-5 stars)

Generate diverse scenarios:
- Some emergency repairs (CM type, completed quickly)
- Regular maintenance (PM type, scheduled in advance)
- Diagnostic visits (PdM type, minimal parts cost)
- Some cancelled appointments with reasons

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments."""

maintenance_order_list = []
def create_maintenance_order_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating client_equipment: {error}")


In [67]:
maintenance_order_list: dict = create_maintenance_order_insertions() #type: ignore

7. Equipment Failures

In [69]:
system_prompt = """You are an excellent JSON data generation assistant, you are going to create a list of equipment failure data with the following constraints:

KEY RELATIONSHIPS:
- maintenance_order_id → maintenance_orders.id (Corrective/CM maintenance orders only)
- failure_type_id → failure_types.id (Specific failure type identified)
- equipment_id → client_equipment.id (Must match equipment from maintenance order)

Business rule:
1. ONLY FOR CORRECTIVE MAINTENANCE:
   - Equipment failures are only recorded for CM (Corrective Maintenance) orders
   - PM (Preventive) and PdM (Predictive) orders should NOT have failure records
   - Maintenance order status should be 'completed'

2. SEVERITY LOGIC:
   - 'Low': Minor issues (1-2 repair hours, <$100 parts)
   - 'Medium': Moderate issues (2-4 repair hours, $100-300 parts)
   - 'High': Serious issues (4-8 repair hours, $300-800 parts)
   - 'Critical': Emergency issues (8+ repair hours, >$800 parts)

3. DATE LOGIC:
   - detected_date: On or before maintenance order scheduled_date
   - resolved_date: Must be same as maintenance order completed_date
   - created_at: Close to when failure was reported/detected

4. FAILURE TYPE MATCHING:
   - Match realistic failure types to equipment
   - Residential equipment: Common failures (capacitors, leaks, filters)
   - Commercial equipment: Complex failures (compressors, control boards, chillers)

5. REPAIR NOTES:
   - Should include: Symptoms, diagnosis, repair performed, parts replaced
   - Use technical but clear language
   - Include preventive recommendations if applicable

Flied Spesifications:
1. Each failure must reference a valid CM maintenance order
2. Equipment must match the maintenance order's equipment
3. Severity_actual should match or be close to failure_type's estimated severity
4. Repair hours should be realistic for the failure type (1-12 hours)
5. 70% should have resolved_date (completed repairs)
6. Repair notes must be detailed and specific
7. Create_at should be within 1-3 days of detected_date

JSON OUTPUT: [
  {
    "id": "EF-001",
    "maintenance_order_id": "string",
    "failure_type_id": "string",
    "equipment_id": "string",
    "detected_date": "YYYY-MM-DD",
    "resolved_date": "YYYY-MM-DD or null",
    "severity_actual": "string (Low/Medium/High/Critical)",
    "repair_notes": "string",
    "repair_hours_actual": "decimal",
    "created_at": "YYYY-MM-DD HH:MM:SS"
  },...
]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = f"""Generate 30 equipment failure records, you can repeate equipment id, in order to create a data historial, but with different fails.

Available Data:
1. MAINTENANCE ORDERS (CM types only): {maintenance_order_list}
2. FAILURE TYPES: {failure_types_list}
3. EQUIPMENT DATA: {client_equipment_list}

GENERATION GUIDELINES:
1. SELECTION:
   - Only use maintenance orders with type_code = 'CM'
   - Only use 'completed' status maintenance orders
   - Match failure types logically to equipment types

2. FAILURE MATCHING LOGIC:
   - Residential Split/Window AC: Capacitor failure, refrigerant leak, dirty coils
   - Commercial Central AC/Chillers: Compressor failure, control board issues, refrigerant leaks
   - Furnaces/Heat Pumps: Ignition problems, heat exchanger issues, reversing valve failure

3. REALISTIC SCENARIOS:
   - Emergency repairs (Critical severity, detected close to scheduled_date)
   - Planned repairs (Lower severity, detected well before scheduled_date)
   - Multiple failures on same equipment (different maintenance orders)
   - Recurring issues on same equipment type

4. DATA CONSISTENCY:
   - equipment_id must match maintenance_order.equipment_id
   - detected_date ≤ maintenance_order.scheduled_date
   - resolved_date = maintenance_order.completed_date (for completed repairs)
   - repair_hours_actual similar to maintenance_order.labor_hours (±20%)

5. REPAIR NOTES EXAMPLES:
   - "Found burnt run capacitor. Replaced with new 45/5 μF capacitor. System tested OK."
   - "Refrigerant leak detected at evaporator coil. Located and repaired leak, recharged system to specification."
   - "Compressor not starting. Diagnosed faulty start capacitor and contactor. Replaced both components."

Generate diverse failure scenarios across different equipment types and severity levels.

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""

equipment_failures_list = []
def create_equipment_failures_insertions() :
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating client_equipment: {error}")

In [70]:
equipment_failures_list: dict = create_equipment_failures_insertions()# type: ignore

8. Inspections

In [72]:
system_prompt = """You are an excellent JSON data generation assistant, you are going to create a list of  equipment inspection records that capture technical measurements and findings.

TECHNICAL PARAMETERS EXPLANATION:
1. pressure_suction: Suction pressure (PSI) - Normal range: 60-85 PSI for R-410A
2. temp_supply: Supply air temperature (°F) - Should be 15-20°F below return temp
3. temp_return: Return air temperature (°F) - Typically 70-80°F
4. amps_reading: Compressor/Blower amps - Compare to nameplate rating

STATUS LOGIC:
- 'Pending': Inspection completed, no decision made yet
- 'Converted to Maintenance': Inspection revealed issues requiring repair
- 'Rejected': Inspection findings acceptable, no action needed

BUSINESS RULES:
1. INSPECTION TYPES:
   - Preventive Maintenance Inspections (routine checks)
   - Diagnostic Inspections (customer-reported issues)
   - Pre-Purchase Inspections (equipment evaluation)

2. TECHNICAL READING RANGES:
   - Normal operation: All readings within normal ranges
   - Marginal operation: 1-2 readings slightly out of range
   - Problematic operation: Multiple readings out of range

3. CONVERSION TO MAINTENANCE:
   - Only 'Converted to Maintenance' status gets converted_to_maintenance_order_id
   - This links to a newly created maintenance order for repairs
   - Typically for CM (Corrective Maintenance) orders

4. TIMELINE:
   - inspection_date: Within last 2 years
   - add new inspection for next year.
   - created_at: Same day as inspection or day before
   - updated_at: After status changes (if converted or rejected)

5. NOTES STRUCTURE:
   - general_notes: What was inspected, visual findings, customer comments
   - findings_summary: Technical analysis, recommendations, severity assessment

Field Spesifications:
1. Match equipment to correct client
2. Assign realistic technical readings based on status:
   - Normal readings for 'Rejected' status
   - Problematic readings for 'Converted to Maintenance' status
   - Mixed readings for 'Pending' status
3. Only provide converted_to_maintenance_order_id for 'Converted to Maintenance' status
4. findings_summary should reference the technical readings
5. Include both residential and commercial equipment inspections
6. 60% should have all technical readings filled
7. Status distribution: 40% Converted, 40% Rejected, 20% Pending

JSON OUTPUT: [
  {
    "id": "INS-001",
    "client_id": "string",
    "equipment_id": "string",
    "user_id": "string",
    "inspection_date": "YYYY-MM-DD",
    "pressure_suction": "decimal or null",
    "temp_supply": "decimal or null", 
    "temp_return": "decimal or null",
    "amps_reading": "decimal or null",
    "general_notes": "string",
    "findings_summary": "string",
    "status": "string (Pending/Converted to Maintenance/Rejected)",
    "converted_to_maintenance_order_id": "string or null",
    "created_at": "YYYY-MM-DD HH:MM:SS",
    "updated_at": "YYYY-MM-DD HH:MM:SS"
  },...
]

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments.
"""
user_prompt = f"""Generate equipment inspection records, based in this list: {maintenance_order_list} .

Available Data:
1. CLIENTS & EQUIPMENT: {all_equipments_by_clients}
2. USERS (Technicians): {technician_id}
3. MAINTENANCE ORDERS (for conversion reference): {maintenance_order_list}

GENERATION GUIDELINES:

1. TECHNICAL READINGS (R-410A Systems):
   - Normal Ranges:
     * pressure_suction: 65-80 PSI (summer), 50-65 PSI (winter)
     * temp_supply: 50-60°F (should be 15-25°F below return)
     * temp_return: 70-80°F (room temperature)
     * amps_reading: 10-20A (check against nameplate)

   - Problem Indicators:
     * Low suction pressure (<60 PSI): Low refrigerant, restriction
     * High suction pressure (>85 PSI): Overcharge, bad compressor
     * Low temp split (<15°F): Low airflow, undercharge
     * High temp split (>25°F): High airflow, overcharge
     * High amps: Dirty coils, failing compressor

2. INSPECTION SCENARIOS:
   - Routine Maintenance Inspection: All readings normal, status 'Rejected'
   - Diagnostic Inspection (Poor Cooling): High suction pressure, low temp split → 'Converted'
   - Seasonal Check-up: Mixed readings, status 'Pending' for review
   - Emergency Diagnostic: Critical readings, immediate 'Converted'

3. NOTES CONTENT:
   general_notes should include:
   - Inspection type and purpose
   - Visual observations (cleanliness, physical condition)
   - Customer-reported symptoms
   - Initial assessments

   findings_summary should include:
   - Analysis of technical readings
   - Specific issues identified
   - Recommendations (repair, replace, monitor)
   - Estimated repair complexity

4. CONVERSION LOGIC:
   - Only convert inspections with clear issues requiring repair
   - converted_to_maintenance_order_id should reference a CM order
   - Conversion typically happens 1-7 days after inspection

5. EXAMPLE SCENARIOS:
   Scenario 1: Normal Operation
   - Readings: 72 PSI, 55°F supply, 75°F return, 12.5A
   - Status: 'Rejected'
   - Notes: "System operating within normal parameters"

   Scenario 2: Refrigerant Leak
   - Readings: 48 PSI, 62°F supply, 78°F return, 8.5A
   - Status: 'Converted to Maintenance'
   - Notes: "Low suction pressure indicates probable refrigerant leak"

   Scenario 3: Dirty Coils
   - Readings: 85 PSI, 58°F supply, 76°F return, 18.5A
   - Status: 'Converted to Maintenance'
   - Notes: "High pressure and amps indicate restricted airflow"

Generate diverse inspection scenarios across different equipment types and seasons.

Output ONLY the JSON array (starting with `[` and ending with `]`). No markdown, no explanations, no comments."""

inspections_list = []
def create_inspection_insertions():
    try:
        deepseek_ai = DeepseekAI(system_prompt, user_prompt, API_KEY, BASE_URL)
        return deepseek_ai.run_AI()
    except Exception as error:
            print(f"Error creating client_equipment: {error}")

In [73]:
inspections_list: dict = create_inspection_insertions()# type: ignore

### Chechking Data

In [75]:
print(f"""
Tables: 
Users: {len(users_list)},
Clients: {len(clients_list)},
Maintenance Types: {len(maintenance_types_list)},
Failures: {len(failure_types_list)},
Clients Equipment: {len(client_equipment_list)},
Maintenance Order: {len(maintenance_order_list)},
Equipment Failures: {len(equipment_failures_list)},
Inspections: {len(inspections_list)},
""")


Tables: 
Users: 22,
Clients: 45,
Maintenance Types: 15,
Failures: 18,
Clients Equipment: 65,
Maintenance Order: 50,
Equipment Failures: 30,
Inspections: 20,



### Creating JSON file.

In [76]:
def Get_JSON(file_name,data):
    """
    Convert a list of product dictionaries into a JSON format.
    
    Args:
        data: List of dictionaries
        
    Returns:
        JSON: JSON formatted string of the product data
    """

    if len(data) <= 0: return print("Data is empty")
    # root
    root = None

    # Make folder
    folder_path = "./json"
    os.makedirs(folder_path, exist_ok=True)

    #file root
    root = os.path.join(f"{folder_path}/{file_name}.json")
    try:
        with open(root, "w") as file:
            json.dump([],file)

        def insert_JSON(data, file_path = root):
            with open(file_path, "r+", encoding= "utf-8") as file:
                file_data = json.load(file)
                file_data.append(data)
                file.seek(0)
                json.dump(file_data, file, indent= 4)

        for object in data:
            insert_JSON(object)

    except Exception as e:
        print(f" JSON | Error: {e}")

In [77]:
Get_JSON("users", users_list)
Get_JSON("clients", clients_list)
Get_JSON("maintenance_types", maintenance_types_list)
Get_JSON("failure_types", failure_types_list)
Get_JSON("client_equipment", client_equipment_list)
Get_JSON("maintenance_order", maintenance_order_list)
Get_JSON("equipment_failures", equipment_failures_list)
Get_JSON("inspections", inspections_list)
print("JSON files created successfully.")

JSON files created successfully.
